In [26]:
import pandas as pd
import numpy as np 
import glob
from biopandas.pdb import PandasPdb
import warnings
warnings.filterwarnings("ignore")
from get_distances import *

In [2]:
phosphosite_data = pd.read_csv("/qfs/projects/proteometer/phospho_site_plus_human_20240422.tsv", delimiter='\t')
pka_data = pd.read_csv("/rcfs/projects/proteometer/all_pka.csv", header=0)

In [3]:
pka_data = pka_data.drop(columns=["Unnamed: 0"])
pka_data

,uniprotID,AA,res_number,pK,state,position
0,A0PJZ3,NTR,5001,6.891614,undefined,5001.0
1,A0PJZ3,LYS,2,10.328119,protonated,2.0
2,A0PJZ3,SER,5,NaN,undefined,5.0
3,A0PJZ3,LYS,6,10.375520,protonated,6.0
4,A0PJZ3,SER,24,NaN,undefined,24.0
...,...,...,...,...,...,...
5916609,P51587,ASP,1397,3.790052,deprotonated,NaN
5916610,P51587,CYS,1398,9.230363,protonated,NaN
5916611,P51587,THR,1399,NaN,undefined,NaN
5916612,P51587,CTR,6400,2.997512,deprotonated,NaN


In [4]:
phosphosite_data


,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,LT_LIT,MS_LIT,MS_CST,CST_CAT#,Ambiguous_Site
0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,NaN,3.0,1.0,NaN,0
1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,NaN,8.0,NaN,NaN,0
2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,NaN,NaN,4.0,NaN,0
3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,NaN,NaN,1.0,NaN,0
4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,NaN,4.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389029,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,ATAAVPTsQSVkHET,NaN,1.0,NaN,NaN,0
389030,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,TRVktPTsQSYR___,NaN,1.0,NaN,NaN,0
389031,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,SDCKLPTsEESRLGS,NaN,1.0,NaN,NaN,0
389032,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,DPFkARVssGyVPPP,1.0,NaN,NaN,NaN,0


In [5]:
# clean phosphosite_data
phosphosite_data['RES_NUM'] = phosphosite_data['MOD_RSD'].str.split('-').str[0].str[1:] # split string on '-' and get rid of first letter
phosphosite_data['PKA_ID'] = phosphosite_data['ACC_ID'] +"_"+ phosphosite_data['RES_NUM']
phosphosite_data


,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,LT_LIT,MS_LIT,MS_CST,CST_CAT#,Ambiguous_Site,RES_NUM,PKA_ID
0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,NaN,3.0,1.0,NaN,0,2,P31946_2
1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,NaN,8.0,NaN,NaN,0,6,P31946_6
2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,NaN,NaN,4.0,NaN,0,21,P31946_21
3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,NaN,NaN,1.0,NaN,0,32,P31946_32
4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,NaN,4.0,NaN,NaN,0,39,P31946_39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389029,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,ATAAVPTsQSVkHET,NaN,1.0,NaN,NaN,0,450,Q9UKY1_450
389030,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,TRVktPTsQSYR___,NaN,1.0,NaN,NaN,0,891,Q9Y2X9_891
389031,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,SDCKLPTsEESRLGS,NaN,1.0,NaN,NaN,0,1196,O15014_1196
389032,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,DPFkARVssGyVPPP,1.0,NaN,NaN,NaN,0,169,Q15942_169


In [6]:
pka_data['PKA_ID'] = pka_data['uniprotID'].str.split('-').str[0] +"_"+ pka_data['res_number'].apply(str)
pka_data

,uniprotID,AA,res_number,pK,state,position,PKA_ID
0,A0PJZ3,NTR,5001,6.891614,undefined,5001.0,A0PJZ3_5001
1,A0PJZ3,LYS,2,10.328119,protonated,2.0,A0PJZ3_2
2,A0PJZ3,SER,5,NaN,undefined,5.0,A0PJZ3_5
3,A0PJZ3,LYS,6,10.375520,protonated,6.0,A0PJZ3_6
4,A0PJZ3,SER,24,NaN,undefined,24.0,A0PJZ3_24
...,...,...,...,...,...,...,...
5916609,P51587,ASP,1397,3.790052,deprotonated,NaN,P51587_1397
5916610,P51587,CYS,1398,9.230363,protonated,NaN,P51587_1398
5916611,P51587,THR,1399,NaN,undefined,NaN,P51587_1399
5916612,P51587,CTR,6400,2.997512,deprotonated,NaN,P51587_6400


In [7]:
len(pka_data['PKA_ID'].unique())

4350682

In [8]:
len(pka_data['uniprotID'].unique())

20595

In [9]:
full_data = pd.merge(phosphosite_data, pka_data, on="PKA_ID", how = "left")
print(full_data.shape)
full_data

(436145, 23)


,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,...,CST_CAT#,Ambiguous_Site,RES_NUM,PKA_ID,uniprotID,AA,res_number,pK,state,position
0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,...,NaN,0,2,P31946_2,P31946,THR,2.0,NaN,undefined,2.0
1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,...,NaN,0,6,P31946_6,P31946,SER,6.0,NaN,undefined,6.0
2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,...,NaN,0,21,P31946_21,P31946,TYR,21.0,11.100927,protonated,21.0
3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,...,NaN,0,32,P31946_32,P31946,THR,32.0,NaN,undefined,32.0
4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,...,NaN,0,39,P31946_39,P31946,SER,39.0,NaN,undefined,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436140,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,ATAAVPTsQSVkHET,...,NaN,0,450,Q9UKY1_450,Q9UKY1,SER,450.0,NaN,undefined,450.0
436141,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,TRVktPTsQSYR___,...,NaN,0,891,Q9Y2X9_891,Q9Y2X9,SER,891.0,NaN,undefined,891.0
436142,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,SDCKLPTsEESRLGS,...,NaN,0,1196,O15014_1196,O15014,SER,1196.0,NaN,undefined,1196.0
436143,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,DPFkARVssGyVPPP,...,NaN,0,169,Q15942_169,Q15942,SER,169.0,NaN,undefined,169.0


In [10]:
full_noduplicates_data = full_data.drop_duplicates()
full_noduplicates_data

,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,...,CST_CAT#,Ambiguous_Site,RES_NUM,PKA_ID,uniprotID,AA,res_number,pK,state,position
0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,...,NaN,0,2,P31946_2,P31946,THR,2.0,NaN,undefined,2.0
1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,...,NaN,0,6,P31946_6,P31946,SER,6.0,NaN,undefined,6.0
2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,...,NaN,0,21,P31946_21,P31946,TYR,21.0,11.100927,protonated,21.0
3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,...,NaN,0,32,P31946_32,P31946,THR,32.0,NaN,undefined,32.0
4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,...,NaN,0,39,P31946_39,P31946,SER,39.0,NaN,undefined,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436140,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,ATAAVPTsQSVkHET,...,NaN,0,450,Q9UKY1_450,Q9UKY1,SER,450.0,NaN,undefined,450.0
436141,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,TRVktPTsQSYR___,...,NaN,0,891,Q9Y2X9_891,Q9Y2X9,SER,891.0,NaN,undefined,891.0
436142,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,SDCKLPTsEESRLGS,...,NaN,0,1196,O15014_1196,O15014,SER,1196.0,NaN,undefined,1196.0
436143,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,DPFkARVssGyVPPP,...,NaN,0,169,Q15942_169,Q15942,SER,169.0,NaN,undefined,169.0


In [11]:
print(pka_data.shape)
print(phosphosite_data.shape)
print(full_noduplicates_data.shape) # 1468 of them didn't match up 
# I guess this makes sense. pka didn't output for ALL residues, and that's not that large of a fraction


(5916614, 7)
(389034, 17)
(436012, 23)


In [12]:
full_noduplicates_data['PKA_ID'].nunique()

370369

In [13]:
full_noduplicates_data['PKA_ID'].value_counts()

PKA_ID
Q8WZ42_1054    91
Q8WZ42_565     89
Q8WZ42_308     87
Q8WZ42_1159    86
Q8WZ42_300     86
               ..
O75592_2348     1
O75592_2344     1
O75592_2342     1
O75592_2338     1
O75592_2701     1
Name: count, Length: 370369, dtype: int64

In [14]:
print(full_data['pK'].notna().sum(), "rows have pka")
print(189514/425877,"have pka annotations")

189765 rows have pka
0.44499702965879817 have pka annotations


In [15]:
# adding the interfaces data
interfaces_data = pd.read_csv("/rcfs/projects/proteometer/ProtVar/predictions/interfaces/2024.05.28_interface_summary_5A.tsv", delimiter='\t', header=0)
interfaces_data

,interaction_id,pdockq,uniprot_id1,uniprot_id2,chain1,chain2,ifresid1,ifresid2,sources,n_references,pdb
0,O75106_Q16853,0.74,O75106,Q16853,A,B,"R169,A203,A204,V205,H206,L212,R213,W220,N226,I...","P39,V209,L218,Q219,W226,N232,I233,S234,G235,A2...","BioGRID,humap,intact,string",2,O75106/O75106_Q16853.pdb
1,Q15118_Q15118,0.73,Q15118,Q15118,A,B,"S53,P54,P56,Y179,D182,R183,M186,L255,A257,H304...","S53,P54,P56,Y179,D182,R183,M186,E253,L255,A257...","BioGRID,intact",2,Q15118/Q15118_Q15118.pdb
2,P11142_Q92598,0.73,P11142,Q92598,A,B,"K25,E27,I28,A30,N31,D32,Q33,G34,R36,E48,L50,D5...","R19,A27,N28,E29,F30,S31,R33,N54,T58,Y184,R261,...","BioGRID,corum,humap,intact,otar,string,xlinkdb",9,P11142/P11142_Q92598.pdb
3,Q13326_Q16585,0.73,Q13326,Q16585,A,B,"V40,L41,L43,L44,L47,V48,N50,L51,T54,I55,L58,F6...","V68,I69,L71,L72,L75,A76,I78,N79,I82,I86,M100,F...","corum,otar,string",0,Q13326/Q13326_Q16585.pdb
4,Q13326_Q92629,0.73,Q13326,Q92629,A,B,"K33,L36,Y37,V40,L41,L43,L44,L47,V48,N50,L51,T5...","R30,K31,C33,L34,F37,V38,L40,L41,L44,I45,V47,N4...","corum,string",0,Q13326/Q13326_Q92629.pdb
...,...,...,...,...,...,...,...,...,...,...,...
486094,P23193_Q92889,0.00,NaN,NaN,NaN,NaN,NaN,NaN,otar,0,NaN
486095,P23193_Q92541,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"BioGRID,intact,otar,string",1,NaN
486096,P23193_Q8WX92,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"otar,string",0,NaN
486097,P23193_Q8WVC0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"BioGRID,intact,otar,string",2,NaN


In [16]:
# adding pockets data
pockets_data = pd.read_csv("/rcfs/projects/proteometer/ProtVar/predictions/pockets/2024.05.28_pockets.tsv", delimiter='\t', header=0)
pockets_data

,struct_id,pocket_id,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_resid,pocket_plddt_mean,pocket_score_combined_scaled
0,A0A024R1R8-F1,1,4.042788,0.316535,0.772959,"{21,22,23,24,25,26,28,29,32}",83.937778,283.034096
1,A0A024R1R8-F1,2,3.175737,0.347111,0.808219,"{12,13,14,15,16,17}",61.206667,102.718057
2,A0A024RBG1-F1,1,7.310256,0.435597,0.856184,"{2,3,4,5,6,7,8,9,10,18,20,21,22,39,40,41,42,47...",89.456190,979.457587
3,A0A024RBG1-F1,2,6.350910,0.389675,0.814896,"{54,57,58,60,61,62,64,65,67,68,69,73,74,75,76,...",83.186923,938.222063
4,A0A024RBG1-F1,3,3.827945,0.378204,0.806045,"{1,2,3,4,5,6,109,110,112,113,114}",77.053636,422.703190
...,...,...,...,...,...,...,...,...
547396,X6R8D5-F1,3,3.894257,0.338401,0.777778,"{86,87,88,89,90,91,92,93,98,100,101,102,103}",56.513846,99.047598
547397,X6R8D5-F1,4,4.196873,0.328247,0.768473,"{43,44,45,46,47,48,49,50,127}",59.902222,107.547205
547398,X6R8D5-F1,5,4.465454,0.314353,0.751790,"{81,87,90,91,92,93,99,100,101,102,103,104,105}",58.298462,95.185313
547399,X6R8D5-F1,6,3.198691,0.398147,0.825342,"{66,67,68,69,71,72,73,75,76,77}",61.416000,122.028350


In [17]:
pockets_data["struct_id"].str.split('-')

0         [A0A024R1R8, F1]
1         [A0A024R1R8, F1]
2         [A0A024RBG1, F1]
3         [A0A024RBG1, F1]
4         [A0A024RBG1, F1]
                ...       
547396        [X6R8D5, F1]
547397        [X6R8D5, F1]
547398        [X6R8D5, F1]
547399        [X6R8D5, F1]
547400        [X6R8D5, F1]
Name: struct_id, Length: 547401, dtype: object

In [18]:
sum(full_noduplicates_data["ACC_ID"].str.contains("F1", case=False, na=False).astype(int))
# we don't have F1 at the end of some of the uniprot-IDs

674

In [19]:
sum(pockets_data["struct_id"].str.contains("F1", case=False, na=False).astype(int))

547401

In [20]:
# getting rid of F1 in the data
pockets_data['uniprot_id'] = pockets_data['struct_id'].str.split('-').str[0]
full_noduplicates_data['uniprot_id'] = full_noduplicates_data['ACC_ID'].str.split('-').str[0]
full_noduplicates_data.drop(columns = ['uniprotID'])

,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,...,CST_CAT#,Ambiguous_Site,RES_NUM,PKA_ID,AA,res_number,pK,state,position,uniprot_id
0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,...,NaN,0,2,P31946_2,THR,2.0,NaN,undefined,2.0,P31946
1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,...,NaN,0,6,P31946_6,SER,6.0,NaN,undefined,6.0,P31946
2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,...,NaN,0,21,P31946_21,TYR,21.0,11.100927,protonated,21.0,P31946
3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,...,NaN,0,32,P31946_32,THR,32.0,NaN,undefined,32.0,P31946
4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,...,NaN,0,39,P31946_39,SER,39.0,NaN,undefined,39.0,P31946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436140,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,ATAAVPTsQSVkHET,...,NaN,0,450,Q9UKY1_450,SER,450.0,NaN,undefined,450.0,Q9UKY1
436141,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,TRVktPTsQSYR___,...,NaN,0,891,Q9Y2X9_891,SER,891.0,NaN,undefined,891.0,Q9Y2X9
436142,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,SDCKLPTsEESRLGS,...,NaN,0,1196,O15014_1196,SER,1196.0,NaN,undefined,1196.0,O15014
436143,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,DPFkARVssGyVPPP,...,NaN,0,169,Q15942_169,SER,169.0,NaN,undefined,169.0,Q15942


In [21]:
pockets_data

,struct_id,pocket_id,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_resid,pocket_plddt_mean,pocket_score_combined_scaled,uniprot_id
0,A0A024R1R8-F1,1,4.042788,0.316535,0.772959,"{21,22,23,24,25,26,28,29,32}",83.937778,283.034096,A0A024R1R8
1,A0A024R1R8-F1,2,3.175737,0.347111,0.808219,"{12,13,14,15,16,17}",61.206667,102.718057,A0A024R1R8
2,A0A024RBG1-F1,1,7.310256,0.435597,0.856184,"{2,3,4,5,6,7,8,9,10,18,20,21,22,39,40,41,42,47...",89.456190,979.457587,A0A024RBG1
3,A0A024RBG1-F1,2,6.350910,0.389675,0.814896,"{54,57,58,60,61,62,64,65,67,68,69,73,74,75,76,...",83.186923,938.222063,A0A024RBG1
4,A0A024RBG1-F1,3,3.827945,0.378204,0.806045,"{1,2,3,4,5,6,109,110,112,113,114}",77.053636,422.703190,A0A024RBG1
...,...,...,...,...,...,...,...,...,...
547396,X6R8D5-F1,3,3.894257,0.338401,0.777778,"{86,87,88,89,90,91,92,93,98,100,101,102,103}",56.513846,99.047598,X6R8D5
547397,X6R8D5-F1,4,4.196873,0.328247,0.768473,"{43,44,45,46,47,48,49,50,127}",59.902222,107.547205,X6R8D5
547398,X6R8D5-F1,5,4.465454,0.314353,0.751790,"{81,87,90,91,92,93,99,100,101,102,103,104,105}",58.298462,95.185313,X6R8D5
547399,X6R8D5-F1,6,3.198691,0.398147,0.825342,"{66,67,68,69,71,72,73,75,76,77}",61.416000,122.028350,X6R8D5


In [22]:
pockets_data['full_id'] = pockets_data['struct_id'] +"_pocket"+ pockets_data['pocket_id'].apply(str)
pockets_data

,struct_id,pocket_id,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_resid,pocket_plddt_mean,pocket_score_combined_scaled,uniprot_id,full_id
0,A0A024R1R8-F1,1,4.042788,0.316535,0.772959,"{21,22,23,24,25,26,28,29,32}",83.937778,283.034096,A0A024R1R8,A0A024R1R8-F1_pocket1
1,A0A024R1R8-F1,2,3.175737,0.347111,0.808219,"{12,13,14,15,16,17}",61.206667,102.718057,A0A024R1R8,A0A024R1R8-F1_pocket2
2,A0A024RBG1-F1,1,7.310256,0.435597,0.856184,"{2,3,4,5,6,7,8,9,10,18,20,21,22,39,40,41,42,47...",89.456190,979.457587,A0A024RBG1,A0A024RBG1-F1_pocket1
3,A0A024RBG1-F1,2,6.350910,0.389675,0.814896,"{54,57,58,60,61,62,64,65,67,68,69,73,74,75,76,...",83.186923,938.222063,A0A024RBG1,A0A024RBG1-F1_pocket2
4,A0A024RBG1-F1,3,3.827945,0.378204,0.806045,"{1,2,3,4,5,6,109,110,112,113,114}",77.053636,422.703190,A0A024RBG1,A0A024RBG1-F1_pocket3
...,...,...,...,...,...,...,...,...,...,...
547396,X6R8D5-F1,3,3.894257,0.338401,0.777778,"{86,87,88,89,90,91,92,93,98,100,101,102,103}",56.513846,99.047598,X6R8D5,X6R8D5-F1_pocket3
547397,X6R8D5-F1,4,4.196873,0.328247,0.768473,"{43,44,45,46,47,48,49,50,127}",59.902222,107.547205,X6R8D5,X6R8D5-F1_pocket4
547398,X6R8D5-F1,5,4.465454,0.314353,0.751790,"{81,87,90,91,92,93,99,100,101,102,103,104,105}",58.298462,95.185313,X6R8D5,X6R8D5-F1_pocket5
547399,X6R8D5-F1,6,3.198691,0.398147,0.825342,"{66,67,68,69,71,72,73,75,76,77}",61.416000,122.028350,X6R8D5,X6R8D5-F1_pocket6


# The procedure:

PSP, pocket, interface

start  with PSP:

loop through unique uniprot id

for each unique uniprot id

select all the PTM sites in the same protein from PSP
select all pockets ..... from procket table
select all interfaces .... from interface table (any pair has uniprot id)

loop through ptm  sites

for each site, 

calculate prockets distance
if site in any pockets: (for loop)
    mindistance = 0
    in pockets = True
    pockets number = [#]
else 
    in pcikates = False
loop through all the pockets
calculate the first taken as minial
       minidistance = #
       pockets number = #
 calcualte any pairt betwene site and pocket residue
 compare the new distance to the minidistant 
  if < 
    replace
    elif =
    append
    
    




- match up uniprot id from pockets data to ptm 
- check if any ptms are located inside of the pocket 
- new column w/ if inside the pocket T/F
-if FALSE, find the closest PTM 


-new column w/ distance from ptm and closest pocket residue

### Pockets Data

In [38]:
# making a smaller test dataset
test_psp = full_noduplicates_data.sample(n = 10)
test_psp['closest_pocket'] = "NaN"
test_psp['inside_pocket'] = 0
test_psp['distance_from_pocket'] = "NaN"
test_psp

,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,...,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_pocket,inside_pocket,distance_from_pocket
202390,SARDH,SARDH,Q9UL12,9q34.2,Y867-p,994515,human,101.04,GCV_T_C,AIDKtIAyGyIHDPS,...,Q9UL12,TYR,867.0,NaN,undefined,867.0,Q9UL12,NaN,0,NaN
416394,NXF1,NXF1,Q9UBU9,11q12.3,R82-m1,27278877,human,70.18,NaN,yttRPNRrGDtWHDR,...,NaN,NaN,NaN,NaN,NaN,NaN,Q9UBU9,NaN,0,NaN
146520,MYO1G,MYO1G,B0I1T2,7p13,S93-p,56649473,human,116.44,Myosin_head,YkAMkHRsRDTCIVI,...,B0I1T2,SER,93.0,NaN,undefined,93.0,B0I1T2,NaN,0,NaN
163077,PABIR1,PABIR1,Q96E09,9q21.11,S173-p,4125437,human,30.53,NaN,SSNGLPPsPIPsPtt,...,Q96E09,SER,173.0,NaN,undefined,173.0,Q96E09,NaN,0,NaN
47792,CKAP4,CKAP4,Q07065,12q23.3,S515-p,27446376,human,66.02,NaN,EQVHtLLsQDQAQAA,...,Q07065,SER,515.0,NaN,undefined,515.0,Q07065,NaN,0,NaN
200975,RYR1,RYR1,P21817,19q13.2,S781-p,7320500,human,565.18,SPRY,GLFFPVVsFSAGVKV,...,P21817,SER,781.0,NaN,undefined,1581.0,P21817,NaN,0,NaN
29478,BOD1L1,BOD1L1,Q8NFC6,4p15.33,T1096-p,3958573,human,330.47,NaN,LAANTLstPSGSSLQ,...,Q8NFC6,THR,1096.0,NaN,undefined,1096.0,Q8NFC6,NaN,0,NaN
167006,PDCD4,PDCD4,Q53EL6,10q25.2,S68-p,487348,human,51.74,NaN,RRLRKNssRDsGrGD,...,Q53EL6,SER,68.0,NaN,undefined,68.0,Q53EL6,NaN,0,NaN
188469,RANBP2,RANBP2,P49792,2q13,S313-p,34357686,human,358.20,NaN,NVQWRALsELAALCY,...,P49792,GLU,313.0,3.989116,deprotonated,1913.0,P49792,NaN,0,NaN
262698,ZEB2,ZEB2,O60315,2q22.3,S327-p,25317408,human,136.45,zf-C2H2,SHSGsYSsHIssKKC,...,O60315,SER,327.0,NaN,undefined,327.0,O60315,NaN,0,NaN


In [40]:
unique_uniprots = test_psp['uniprot_id'].unique() # get all of the unique uniprots

# for each unique uniprotID...
for uniprot in unique_uniprots:
    # isolate to psp and pockets in each uniprot
    psp_only_uniprot = test_psp[test_psp.uniprot_id == uniprot]
    pocket_only_uniprot = pockets_data[pockets_data.uniprot_id == uniprot]


    # parse your structure here
    pdb_path = "/rcfs/projects/proteometer/alphafold_swissprot_pdb"
    pdb_name = glob.glob("/rcfs/projects/proteometer/alphafold_swissprot_pdb/*" + uniprot + "*")
    print("name of pdb is:", pdb_name)
    if pdb_name:  
        ppdb = PandasPdb()  
        ppdb.read_pdb(pdb_name[0])


    # for each psp
        for phosphosite_row_index in psp_only_uniprot.index:
            #print(psp_only_uniprot)
            #print(phosphosite_row_index)
            residue_num = psp_only_uniprot.loc[phosphosite_row_index,'res_number'] # finding the residue number of the psp
            #print(residue_num)
            # use the residue # to get the coordinates in space from pdb file
            
            
            for pocket_index in pocket_only_uniprot.index : # get all the residues in all of the pockets 
                pocket_residues = pocket_only_uniprot.loc[pocket_index,'pocket_resid']

                # check if it's inside of a pocket
                pocket_residues = pocket_residues[1:-1].split(",") # format the pocket_residues because it's a string
                #print(pocket_residues)
                if residue_num in pocket_residues:
                    test_psp.loc[phosphosite_row_index,'inside_pocket'] = 1 # if residue is in the pocket, put 1 in the inside pocket column
                    test_psp.loc[phosphosite_row_index,'closest_pocket'] = pocket_only_uniprot.loc[pocket_index,'full_id'] # put unique pocketID in closest pocket
                    test_psp.loc[phosphosite_row_index,'distance_from_pocket'] = 0 
                    break # break because you don't want to contiue looking for pockets (and therefore overwrite the inside pocket and closest pocket)

            if test_psp.loc[phosphosite_row_index,'inside_pocket'] == 0: # if the phosphosite isn't in any pockets
                print("phosphosite isn't in any pockets")
                min_dist = 100000000000000000000000000000000 # make min dist extremely high at first
                for pocket_index in pocket_only_uniprot.index:
                    input_struct = ppdb.df['ATOM']
                    #print(input_struct)
                    new_dist = find_mean_distances(input_struct, residue_num, pocket_residues)
                    if residue_num:
                        if min_dist > new_dist: # if this is the smallest distance so far, replace min_dist with new_dist
                            test_psp.loc[phosphosite_row_index,'closest_pocket'] = pocket_only_uniprot.loc[pocket_index,'full_id'] # put unique pocketID in closest pocket
                            test_psp.loc[phosphosite_row_index,'distance_from_pocket'] = new_dist # replace distance_from_pocket with min_dist
                            min_dist = new_dist 
                            print("added smallest distance:", min_dist)
                
    else: # if we can't find the pdb file
        for phosphosite_row_index in psp_only_uniprot.index:
            residue_num = str(psp_only_uniprot.loc[phosphosite_row_index,'res_number']) # finding the residue number of the psp
            # use the residue # to get the coordinates in space from pdb file
            
            for pocket_index in pocket_only_uniprot.index : # get all the residues in all of the pockets 
                pocket_residues = pocket_only_uniprot.loc[pocket_index,'pocket_resid']

                # check if it's inside of a pocket
                pocket_residues = pocket_residues[1:-1].split(",") # format the pocket_residues because it's a string
                if residue_num in pocket_residues:
                    # fill all with NaN bc we can't find a pdb file
                    test_psp.loc[phosphosite_row_index,'inside_pocket'] = 'NaN' 
                    test_psp.loc[phosphosite_row_index,'closest_pocket'] = 'NaN' 
                    test_psp.loc[phosphosite_row_index,'distance_from_pocket'] = 'NaN'

        

                


      

name of pdb is: ['/rcfs/projects/proteometer/alphafold_swissprot_pdb/AF-Q9UL12-F1-model_v4.pdb']
phosphosite isn't in any pockets
added smallest distance: 38.85048559245363
name of pdb is: ['/rcfs/projects/proteometer/alphafold_swissprot_pdb/AF-Q9UBU9-F1-model_v4.pdb']
phosphosite isn't in any pockets


IndexError: index 0 is out of bounds for axis 0 with size 0

In [35]:
test_psp

,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,...,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_pocket,inside_pocket,distance_from_pocket
7161,AHNAK,AHNAK,Q09666,11q12.3,S637-p,7990610,human,629.10,NaN,KMKMPtFstPGAKGE,...,Q09666,SER,637.0,NaN,undefined,637.0,Q09666,NaN,0,NaN
387546,SVEP1,SVEP1,Q4LDE5,9q31.3,K789-ub,964548316,human,390.17,NaN,TEWPDCAkKRFANHG,...,Q4LDE5,SER,789.0,NaN,undefined,2989.0,Q4LDE5,NaN,0,NaN
13547,ANK3,ANK3,Q12955,10q21.2,S4290-p,34505252,human,480.41,NaN,LKPKIHGsGHVEEPA,...,NaN,NaN,NaN,NaN,NaN,NaN,Q12955,NaN,0,NaN
293520,SP100,SP100,P23497,2q37.1,K430-ac,41519854,human,100.42,NaN,LRSKHGEkAPMTSRS,...,P23497,LYS,430.0,11.265550,protonated,430.0,P23497,NaN,0,NaN
323898,DMXL1,DMXL1,Q9Y485,5q23.1,K1062-ub,573791626,human,337.84,NaN,NRLAVAYkQPASNSR,...,Q9Y485,ASP,1062.0,4.146039,deprotonated,1462.0,Q9Y485,NaN,0,NaN
302716,ALDH18A1,ALDH18A1,P54886,10q24.1,K662-ub,21128714,human,87.30,NaN,TFsPsEVkSLRTEYG,...,P54886,LYS,662.0,10.429140,protonated,662.0,P54886,NaN,0,NaN
63722,PRKDC,DNAPK,P78527,8q11.21,S841-p,7445648,human,469.09,DNA-PKcs_N,LKKTkNLssNEAIsL,...,P78527,SER,841.0,NaN,undefined,841.0,P78527,NaN,0,NaN
238283,TTN,Titin,Q8WZ42,2q31.2,Y10703-p,3971836,human,3816.03,NaN,EEYEEREyERyEEHE,...,NaN,NaN,NaN,NaN,NaN,NaN,Q8WZ42,NaN,0,NaN
422528,UBAP2,UBAP2,Q5T6F2,9p13.3,R132-m1,41477490,human,117.12,NaN,EkESSrGrGNNNRKG,...,NaN,NaN,NaN,NaN,NaN,NaN,Q5T6F2,NaN,0,NaN
252091,USE1,USE1L,Q9NZ43,19p13.11,S15-p,3996453,human,29.37,Use1,LNLVRLLsRCEAMAA,...,Q9NZ43,SER,15.0,NaN,undefined,15.0,Q9NZ43,NaN,0,NaN


In [31]:
test_psp["inside_pocket"].dtype

dtype('int64')